In [ ]:
!pip install ultralytics opencv-python colorthief gtts librosa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.8/750.8 kB 5.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

Importing Dependancies

In [ ]:
from ultralytics import YOLO
from colorthief import ColorThief
import cv2
import numpy as np
import google.generativeai as genai
from google.colab import userdata
from gtts import gTTS
from IPython.display import display, Audio
import time
import librosa

In [ ]:
ymodel = YOLO('yolov8n.pt')

# Run batched inference on a list of images
def get_predictions(filename):
  results = ymodel.predict(filename, save=True)  # return a list of Results objects
  return results

100%|██████████| 6.23M/6.23M [00:00<00:00, 23.3MB/s]


In [ ]:
def speak(s):
    tts = gTTS(s, tld='co.in')
    tts.save('temp.wav')
    sound_file = 'temp.wav'
    audio = Audio(sound_file, autoplay=True)
    display(audio)
    duration = librosa.get_duration(path='temp.wav')
    time.sleep(duration + 0.5)

In [ ]:
def get_objects(results, flag=0):
  objects = {}
  object_names = []

  for result in results:

      detection_count = result.boxes.shape[0]

      for i in range(detection_count):
          cls = int(result.boxes.cls[i].item())
          name = result.names[cls]
          confidence = float(result.boxes.conf[i].item())
          bounding_box = result.boxes.xyxy[i].cpu().numpy()

          x = int(bounding_box[0])
          y = int(bounding_box[1])
          width = int(bounding_box[2] - x)
          height = int(bounding_box[3] - y)

          object_names.append(name)

          objects[name] = 1 + objects.get(name, 0)
  object_list = []
  for obj in objects:
    object_list.append(str(objects[obj])+" "+obj)

  vision = ",".join(object_list)

  dangers = ["train", "bus", "truck", "motorcycle", "car", "stop sign", "dog"]

  if not flag:
    if "traffic light" in vision:
        traffic_model = YOLO('best.onnx')
        t_results = traffic_model.predict(filename, save=True)
        res, objs = get_objects(t_results, 1)
        if "green_pedestrian_light" in res:
          print("There is a green pedstrian light on the crossing")
          speak("There is a green pedstrian light on the crossing")
        elif "red_pedestrian_light" in res:
          print("LOOK OUT! There is a red pedstrian light on the crossing")
          speak("LOOK OUT! There is a red pedstrian light on the crossing")

        else:
          print("LOOK OUT! There is a traffic light ahead")
          speak("LOOK OUT! There is a traffic light ahead")

    for danger in dangers:
      if danger in vision:
        print("LOOK OUT! There is a "+danger+" in sight")
        speak("LOOK OUT! There is a "+danger+" in sight")

  return vision, object_names

Ask for user's name

In [ ]:
print("May I know your name: ")
speak("May I know your name: ")
name = input()

May I know your name: 


Nishanth


Input image

In [ ]:
filename = input("Enter file name: ")
results = get_predictions(filename)
vision, object_names = get_objects(results)

Enter file name: signal1.jpg

image 1/1 /content/signal1.jpg: 448x640 3 cars, 2 traffic lights, 342.2ms
Speed: 18.0ms preprocess, 342.2ms inference, 2750.6ms postprocess per image at shape (1, 3, 448, 640)
Results saved to runs/detect/predict
WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Loading best.onnx for ONNX Runtime inference...
requirements: Ultralytics requirements ['onnx', 'onnxruntime'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 82.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 191.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 148.7 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 13.6s, installed 2 packages: ['onnx', 'onnxruntime']
requirements: ⚠️ Resta

LOOK OUT! There is a car in sight


Test objects

In [ ]:
object_names

['car', 'car', 'traffic light', 'traffic light', 'car']

In [ ]:
gemini_api_secret_name = 'api1'

try:
  GOOGLE_API_KEY=userdata.get(gemini_api_secret_name)
  genai.configure(api_key=GOOGLE_API_KEY)
except userdata.SecretNotFoundError as e:
   print(f'''Secret not found\n\nThis expects you to create a secret named {gemini_api_secret_name} in Colab\n\nVisit https://makersuite.google.com/app/apikey to create an API key\n\nStore that in the secrets section on the left side of the notebook (key icon)\n\nName the secret {gemini_api_secret_name}''')
   raise e
except userdata.NotebookAccessError as e:
  print(f'''You need to grant this notebook access to the {gemini_api_secret_name} secret in order for the notebook to access Gemini on your behalf.''')
  raise e
except Exception as e:
  print(f"There was an unknown error. Ensure you have a secret {gemini_api_secret_name} stored in Colab and it's a valid key from https://makersuite.google.com/app/apikey")
  raise e

In [ ]:
def count_money():
  money_model = YOLO('money.onnx')
  results = money_model.predict(filename, save=True)

  objects = {}
  object_names = []

  for result in results:

      detection_count = result.boxes.shape[0]

      for i in range(detection_count):
          cls = int(result.boxes.cls[i].item())
          name = result.names[cls]
          confidence = float(result.boxes.conf[i].item())
          bounding_box = result.boxes.xyxy[i].cpu().numpy()

          x = int(bounding_box[0])
          y = int(bounding_box[1])
          width = int(bounding_box[2] - x)
          height = int(bounding_box[3] - y)

          object_names.append(name)

          objects[name] = 1 + objects.get(name, 0)
  object_list = []
  total = 0

  mappings = {"100IndianRupees": 100, "10IndianRupees": 10, "200IndianRupees": 200, "20IndianRupees": 20, "500IndianRupees": 500, "50IndianRupees": 50, "5IndianRupees": 5}
  for obj in objects:
    object_list.append(str(objects[obj])+": "+str(mappings[obj]))
    total +=  mappings[obj]
  vision = ",".join(object_list)
  if len(vision)==0:
    out = "There is no money in sight"
  else:
    out = "The total amount is "+str(total)+" Rupees"
  print(out)
  speak(out)


In [ ]:
def get_crop():
  img = cv2.imread(filename)

  # Extract bounding boxes
  boxes = results[0].boxes.xyxy.tolist()

  # Iterate through the bounding boxes
  for i, box in enumerate(boxes):
      x1, y1, x2, y2 = box
      # Crop the object using the bounding box coordinates
      ultralytics_crop_object = img[int(y1):int(y2), int(x1):int(x2)]
      # Save the cropped object as an image
      cv2.imwrite('ultralytics_crop_' + object_names[i] + '.jpg', ultralytics_crop_object)

def euclidean_distance(color1, color2):
    return np.sqrt(np.sum((np.array(color1) - np.array(color2))**2))

def get_color(color_object):
  color_thief = ColorThief('ultralytics_crop_'+color_object+".jpg")
  rgb_tuple = color_thief.get_color(quality=1)

  color_names_rgb = {
      'White': (255, 255, 255),
      'Black': (0, 0, 0),
      'Red': (255, 0, 0),
      'Green': (0, 255, 0),
      'Blue': (0, 0, 255),
      'Yellow': (255, 255, 0),
      'Cyan': (0, 255, 255),
      'Magenta': (255, 0, 255),
      'Gray': (128, 128, 128),
      'Maroon': (128, 0, 0),
      'Olive': (128, 128, 0),
      'Lime': (0, 128, 0),
      'Teal': (0, 128, 128),
      'Navy': (0, 0, 128),
      'Purple': (128, 0, 128),
      'Pink': (255, 20, 147),
  }
  min_distance = float('inf')
  nearest_color_name = None
  for name, color in color_names_rgb.items():
      distance = euclidean_distance(rgb_tuple, color)
      if distance < min_distance:
          min_distance = distance
          nearest_color_name = name
  return nearest_color_name

In [ ]:
#@title Interact with Gemini
print(f"GEMINI: Hello {name}, I'll be your vision today")
speak(f"Hello {name}, I'll be your vision today")

print("GEMINI: You have "+vision+" in your surroundings")
speak("You have "+vision+" in your surroundings")

text: str = "There are "+vision+" where am i?"

gmodel = genai.GenerativeModel('gemini-pro')
chat = gmodel.start_chat(history=[])

response = chat.send_message('%s -- Please answer as concisely as you can, avoiding any extra conversation or text' % text)
gemini_response = response.text

print("GEMINI: You are at an "+ gemini_response)
speak("You are at "+ gemini_response)

while(True):
  text = input("Me: ")
  if text=="Stop":
    break
  elif "color" in text:
    color_object = text.split()[-1]
    get_crop()
    try:
      out = "The color of the " + color_object + " is " + get_color(color_object)
      print(out)
      speak(out)
    except Exception:
      speak("Invalid Object Name")
  elif "money" in text:
    count_money()
  else:
    response = chat.send_message(text)
    gemini_response = response.text
    print("GEMINI:", gemini_response)
    speak(gemini_response)

GEMINI: Hello Nishanth, I'll be your vision today


GEMINI: You have 3 car,2 traffic light in your surroundings


GEMINI: You are at an In a Traffic Intersection


Me: How much money do I have
WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Loading money.onnx for ONNX Runtime inference...

image 1/1 /content/signal1.jpg: 640x640 (no detections), 331.4ms
Speed: 8.2ms preprocess, 331.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict4
There is no money in sight


Me: How many cars are there
GEMINI: 3 cars


Me: Stop


Re-run

In [ ]:
filename = input("Enter file name: ")
results = get_predictions(filename)
vision, object_names = get_objects(results)

Enter file name: signal1.jpg

image 1/1 /content/signal1.jpg: 448x640 3 cars, 2 traffic lights, 209.0ms
Speed: 4.2ms preprocess, 209.0ms inference, 1.5ms postprocess per image at shape (1, 3, 448, 640)
Results saved to runs/detect/predict
WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Loading best.onnx for ONNX Runtime inference...

image 1/1 /content/signal1.jpg: 640x640 1 red_pedestrian_light, 3 traffic_lights, 225.4ms
Speed: 7.1ms preprocess, 225.4ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict3
LOOK OUT! There is a red pedstrian light on the crossing


LOOK OUT! There is a car in sight


In [ ]:
#@title Gemini re-run
gmodel = genai.GenerativeModel('gemini-pro')
chat = gmodel.start_chat(history=[])

response = chat.send_message('You are a virtual assistant to help the blind people, you will be given a set of objects in the surroundings. Your job is to answer questions from the  please answer as concisely as you can, avoiding any extra conversation or text')
gemini_response = response.text

# print("GEMINI: You are at an "+ gemini_response)
# speak("You are at "+ gemini_response)

while(True):
  text = input("Me: ")
  if text=="Stop":
    break
  elif "color" in text:
    color_object = text.split()[-1]
    get_crop()
    try:
      out = "The color of the " + color_object + " is " + get_color(color_object)
      print(out)
      speak(out)
    except Exception:
      speak("Invalid Object Name")
  elif "money" in text:
    count_money()
  else:
    response = chat.send_message(text)
    gemini_response = response.text
    print("GEMINI:", gemini_response)
    speak(gemini_response)

Me: What is the color of the car
The color of the car is Black


Me: Stop
